# Выбор региона нефтедобычи

### Введение
Нефтедобывающая компания должна решить, где бурить новые скважины.
Имеются пробы нефти в трёх регионах, в каждом — 100 000 месторождений, где измерили качество нефти и объём её запасов.   
Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Условия задачи:

•	Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).  
•	При разведке региона проводится исследование 500 точек.  
•	Бюджет на разработку месторождений — 10 млрд рублей, стоимость бурения одной скважины — 50 млн рублей.  
•	Один баррель сырья приносит 4500 рублей прибыли.  
•	Не рассматривать регионы, в которых риск убытков выше 2.5%. Из оставшихся выбирается регион с наибольшей средней прибылью.


Описание данных  

•	id — уникальный идентификатор месторождения;  
•	f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);  
•	product — объём запасов в месторождении (тыс. баррелей). 

План работы
1. Подготовка данных.  
2. Обучение и проверка модели для каждого региона.  
3. Подготовка к расчёту прибыли.   
4. Оценка риска и прибыли для каждого региона.
5. Общий вывод.

### 1. Подготовка данных

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np

In [2]:
# загрузка данных
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')

Проверка данных по каждому региону.

In [3]:
regions = {'Регион_0': df0, 'Регион_1': df1, 'Регион_2': df2}
for reg, data in regions.items():
    print (reg, '   Первые пять строк базы данных')
    print ()
    print (data.head())
    print ()
    print (reg, '   Общая информация - проверка на наличие пропусков, проверка типа данных')
    print ()
    print (data.info())
    print ()
    print (reg, '   Количество дупликатов строк:',data.duplicated().sum())
    print ()

Регион_0    Первые пять строк базы данных

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

Регион_0    Общая информация - проверка на наличие пропусков, проверка типа данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Регион_0    Количество дупликатов строк: 0

Регион_1    Первые пять строк базы данных

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  

Поиск дупликатов строк в базах данных разных регионов - проверка на наличие ошибок при формировании баз данных:

In [4]:
pd.concat([df0,df1,df2]).duplicated().sum()

0

Удалим столбец id - он не нужен в данной работе.

In [5]:
df0 = df0.drop(columns=['id'])
df1 = df1.drop(columns=['id'])
df2 = df2.drop(columns=['id'])

#### Вывод по предобработке данных   

Пропуски в данных не обнаружены. Дупликаты не обнаружены. Изменение типа данных не требуется.  
Из таблиц удалили столбцы, не имеющие отношения к данной работе.

### 2. Обучение и проверка модели

Значения признаков и их разбросы в разных столбцах различаются значительно. Поэтому сначала произведем масштабирование признаков.   
Чтобы не допустить утечки валидационных данных в обучающую выборку, настроим scaler по обучающей выборке, а затем уже применим к отдельно к каждой выборке базе.

Для создания модели воспользуемся алгоритмом линейной регрессии.

Предсказания и правильные ответы сохраним в виде series с согласованными индексами. Это понадобится ниже для расчета прибыли.

Рассчитаем средний запас сырья в одной скважине каждого региона и корень из средней квадратичной ошибки модели (RMSE).  
Дополнительно, чтобы иметь представление об однородности скважин региона, найдем разброс значений запасов сырья и стандартное отклонение.   
Данные представим в виде таблицы.

In [6]:
# импорт функции train_test_split библиотеки sklearn
from sklearn.model_selection import train_test_split

In [7]:
# импорт функции масштабирования
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# список признаков, которые будем масштабировать
numeric = ['f0','f1','f2']

In [8]:
# импорт алгоритма линейной регрессии
from sklearn.linear_model import LinearRegression

In [9]:
# импорт функции расчета СКО (MSE) 
from sklearn.metrics import mean_squared_error as mse

In [10]:
# чтобы избежать дубликации кода, создадим функцию model, которая будет производить
# разбивку датасета, масштабирование, обучение модели, получение предсказаний, расчёт RMSE и других величин

def model(df):
    # выделение обучающей и валидационной выборок
    df_train, df_valid = train_test_split(df, test_size=0.25, random_state=12345)
    
    # масштабирование признаков
    scaler.fit(df_train[numeric]) # настраиваем scaler по обучающей выборке 
    # применяем scaler
    df_train[numeric] = scaler.transform(df_train[numeric])
    df_valid[numeric] = scaler.transform(df_valid[numeric])
    
    # создание и обучение модели
    model = LinearRegression()
    features_train = df_train.drop(columns='product')
    target_train = df_train['product']
    model.fit(features_train, target_train)
    
    # предсказания и правильные ответы
    features_valid = df_valid.drop(columns='product')
    target_valid = df_valid['product'].reset_index(drop=True) # правильные ответы
    pred_valid = pd.Series(model.predict(features_valid)) # предсказания
    
    # средний запас сырья в одной скважине каждого региона и корень из средней квадратичной ошибки модели (RMSE).  
    # дополнительно, чтобы иметь представление об однородности скважин региона, найдем разброс значений запасов сырья
    # и стандартное отклонение.
    mean = round(df['product'].mean(),1) # cредний запас сырья
    rmse = round(mse(target_valid,pred_valid)**0.5,1) # кв.корень из СКО (RMSE) модели
    ran = round(df['product'].max()-df['product'].min(),1) # разброс значений
    sd = round(np.std(df['product']),1) # cтандартное отклонение
    
    # функция возвращает фактические и прогнозные значения по объемам нефти,
    # средний запас сырья в одной скважине каждого региона, RMSE, 
    # разброс значений запасов сырья и стандартное отклонение
    return target_valid, pred_valid, mean, rmse, ran, sd

In [11]:
import warnings
warnings.simplefilter("ignore")

# применим функцию model к регионам
target_valid0, pred_valid0, mean0, rmse0, ran0, sd0 = model(df0)
target_valid1, pred_valid1, mean1, rmse1, ran1, sd1 = model(df1)
target_valid2, pred_valid2, mean2, rmse2, ran2, sd2 = model(df2)

In [12]:
# заготовка таблицы
columns1 = ['region','mean_volume', 'RMSE', 'range', 'standard_dev']
line0 = ['region_0', mean0, rmse0, ran0, sd0]
line1 = ['region_1', mean1, rmse1, ran1, sd1]
line2 = ['region_2', mean2, rmse2, ran2, sd2]
data1 = [line0, line1, line2]

In [13]:
# таблица искомых значений
summary1 = pd.DataFrame(data=data1, columns=columns1)
summary1

,region,mean_volume,RMSE,range,standard_dev
0,region_0,92.5,37.6,185.4,44.3
1,region_1,68.8,0.9,137.9,45.9
2,region_2,95.0,40.0,190.0,44.7


#### Анализ результата работы модели.

Модель показывает исключительно хорошие результаты для региона 1. Для регионов 0 и 2 RМSE модели (т.е., фактически, ошибка предсказания) составляет более 40% от величины среднего запаса сырья в скважине. Это может быть связано с природой признаков f0, f1 и f2 и их применимостью для конкретного региона.   
Отметим, что скважины каждого региона очень неоднородны по количеству запасов - это видно по разбросам значений и величине стандартного отклонения.

### 3. Подготовка к расчёту прибыли

### 3.1. Ключевые значения для расчётов.

In [14]:
# количество точек (points of research), исследуемых при разведке
p = 500

In [15]:
# бюджет (budget) на разработку месторождения, млн руб.
b = 10000

In [16]:
# стоимость бурения одной скважины (investments per well), млн руб.
ipw = 50

In [17]:
# количество скважин исходя из бюджета
w = int(b/ipw)
w

200

In [18]:
# прибыль с одной тысячи баррелей (profit per kilo barrel), млн руб.
pkb = 4500*1000/1000000
pkb

4.5

In [19]:
# приемлимый риск убытков (acceptable risk), процент
risk_accept = 2.5

### 3.2. Минимальный средний объём сырья в месторождениях региона, достаточный для его разработки.

In [20]:
# среднее количество сырья в одной скважине (volume per well), необходимое для компенсации затрат на ее бурение, тыс. барр.
vpw = round (ipw / pkb, 1)
vpw

11.1

#### Вывод
Одна скважина должна дать в среднем не менее 11.1 тыс. баррелей нефти, чтобы покрыть расходы на бурение.

### 3.3. Функция для расчёта прибыли по набору отобранных месторождений и предсказаний модели.

In [21]:
# Функция принимает на вход объемы сырья (тыс. баррелей) в каждой скважине и количество скважин в выборке;
# возвращает суммарную валовую прибыль (млн руб.) со всех этих скважин.

def prof_reg (target, pred, n): 
    pred_sorted = pred.sort_values(ascending=False) # предсказанные объемы сырья, сортированные по убыванию
    target_selected = target[pred_sorted.index].head(n) # выборка n фактических объемом, соответствующиих макс. предсказанным
    income = target_selected.sum()*pkb # фактическая выручка с n отобранных скважин, млн руб.
    ips = ipw*n # стоимость бурения n скважин (investment per sample), млн руб.
    profit = income-ips # валовая прибыль с n отобранных скважин, млн руб.
    return profit

### 4.	Риски и прибыль для каждого региона.

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. За меру риска убытков примем процент отрицательных значений profit.  
Результат представим в виде таблицы.

In [22]:
# импорт необходимых функий
from scipy import stats as st
from numpy.random import RandomState # эта функция необходима для применения метода bootstrap
state = RandomState(12345) 

In [23]:
# заготовка таблицы
columns=['region','mean_profit','95%_low', '95%_high', '2.5%_quantile', 'risk_%', 'risk_status']
data=[]

In [24]:
regions = {'region_0':[target_valid0, pred_valid0], 'region_1': [target_valid1, pred_valid1], 'region_2': [target_valid2, pred_valid2]}
for reg, tp in regions.items():
    values = []
    for i in range(1000): # техника bootsprap для нахождения распределения прибыли
        target_subsample = tp[0].sample(n=p, replace=True, random_state = state) # выборка p скважин (p=500)
        pred_subsample = tp[1][target_subsample.index] # предсказанные объемы сырья в выбранных скважинах
        values.append(prof_reg(target_subsample, pred_subsample, w)) # расчет фактической прибыли с данной выборки
    values = pd.Series(values)
    mean = values.mean() # среднее значение прибыли, млн
    ci = st.t.interval(0.95, len(values)-1,loc=mean, scale=values.sem()) # доверительный интервал
    q = values.quantile(0.025).astype('int64') # 2.5% квантиль
    values_n = values[values<0] # поиск негативных значений прибыли
    risk = round(len(values_n)/len(values)*100,1) # расчет доли негативных значений прибыли
    if risk < risk_accept: # проверка критерия риска
        risk_status = 'OK'
    else:
        risk_status = 'NG'
    data.append([reg, mean.astype('int64'), ci[0].astype('int64'), ci[1].astype('int64'), q, risk, risk_status])

In [25]:
# результаты по всем регионам
summary = pd.DataFrame(data=data, columns=columns)
summary

,region,mean_profit,95%_low,95%_high,2.5%_quantile,risk_%,risk_status
0,region_0,94259,94087,94431,88979,0.0,OK
1,region_1,95182,95052,95312,91281,0.0,OK
2,region_2,94201,94025,94378,88841,0.0,OK


In [26]:
# выбор региона исходя из среднего значения прибыли
best = summary[summary['mean_profit']==summary['mean_profit'].max()]
best

,region,mean_profit,95%_low,95%_high,2.5%_quantile,risk_%,risk_status
1,region_1,95182,95052,95312,91281,0.0,OK


Т.о., для дальнейшей работы рекомендован регион 1: суммарная валовая прибыль с отобранных скважин - 95182 млн руб., с 95% вероятностью значение прибыли лежит в пределах 95052-95312 млн руб., риск убыточности отсутствует.

### 5. Общий вывод.

Была проделана следующая работа:  
Данные были проверены на необходимость предобработки и масшабированы.   
Для каждого региона были созданы и обучены модели на основе алгоритма линейной регрессии.
Вычислены значения RMSE, средние значения запасов сырья в скважинах, минимальное безубыточное значение сырья в одной скважине.  
С помощью техники bootstrap вычислена суммарная валовая прибыль с отобранных скважин в каждом районе, 95% доверительный интервал ее значений и оценен риск убыточности.  
В результате пришли к заключению, что для дальнейшей работы рекомендован регион 1: суммарная валовая прибыль с отобранных скважин - 95182 млн руб., с 95% вероятностью значение прибыли лежит в пределах 95052-95312 млн руб., риск убыточности отсутствует.